In [1]:
import plaidml.keras
plaidml.keras.install_backend()
import os
os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"

In [2]:
# Importing useful libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, GRU, Bidirectional, Conv1D, Flatten, MaxPooling1D
from keras.optimizers import SGD
import math
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from keras import optimizers

import time 

### Data Processing

In [3]:
df = pd.read_csv('../data/num_data.csv')

In [4]:
dataset = df

In [5]:
dataset.shape

(420768, 16)

In [6]:
# Useful functions
def plot_test_pred(test, predicted):
    plt.figure(figsize=(30, 15));

    plt.plot(test, color='red', alpha=0.5, label='Actual PM2.5 Concentration',)
    plt.plot(predicted, color='blue', alpha=0.5, label='Predicted PM2.5 Concentation')
    plt.title('PM2.5 Concentration Prediction')
    plt.xlabel('Time')
    plt.ylabel('PM2.5  Concentration')
    plt.legend()
    plt.show()
    

def return_rmse(test,predicted):
    rmse = math.sqrt(mean_squared_error(test, predicted))
    return rmse

In [7]:
data_size = dataset.shape[0]
train_size=int(data_size * 0.6)
test_size = 100
valid_size = data_size - train_size - test_size

test_next_day = [12, 24, 48]

In [8]:
training_set = dataset[:train_size].values
valid_set = dataset[train_size:train_size+valid_size].values
test_set = dataset[data_size-test_size:].values

In [9]:
y = dataset.iloc[:,4].values
y = y.reshape(-1,1)
n_feature = training_set.shape[1]
y.shape

(420768, 1)

In [10]:
n_feature

16

In [11]:
# Scaling the dataset
sc = MinMaxScaler(feature_range=(0,1))
training_set_scaled = sc.fit_transform(training_set)
valid_set_scaled = sc.fit_transform(valid_set)
test_set_scaled = sc.fit_transform(test_set)

sc_y = MinMaxScaler(feature_range=(0,1))
y_scaled = sc_y.fit_transform(y)

In [12]:
# convert dataset into sequences, where n_steps_in is the input sequence lengith, 
# and n_steps_out is the output sequence length 
def convert_to_sequences(sequences, n_steps_in, n_steps_out):
    X_, y_ = [], []
    for i in range(len(sequences)):
        tail_x = i + n_steps_in
        out_tail_x = tail_x + n_steps_out-1
        if out_tail_x > len(sequences):
            break
        seq_x, seq_y = sequences[i:tail_x, :], sequences[tail_x-1:out_tail_x, 0]
        X_.append(seq_x)
        y_.append(seq_y)
    return np.array(X_), np.array(y_)

In [13]:
n_steps_in = 24
n_steps_out = 24
X_train, y_train = convert_to_sequences(training_set_scaled, n_steps_in, n_steps_out)
X_valid, y_valid = convert_to_sequences(valid_set_scaled, n_steps_in, n_steps_out)
X_test, y_test = convert_to_sequences(test_set_scaled, n_steps_in, n_steps_out)

In [14]:
GRU_reg = Sequential()
LSTM_reg = Sequential()
GRU_GRU_reg =  Sequential()
GRU_LSTM_reg = Sequential()
LSTM_GRU_reg = Sequential()
LSTM_LSTM_reg = Sequential()


GRU_reg.add(GRU(units=50, input_shape=(X_train.shape[1],n_feature), activation='tanh'))
# The output layer
GRU_reg.add(Dense(units=n_steps_out))


LSTM_reg.add(LSTM(units=50, input_shape=(X_train.shape[1],n_feature), activation='tanh'))
LSTM_reg.add(Dense(units=n_steps_out))


GRU_GRU_reg.add(GRU(units=50, return_sequences=True, input_shape=(X_train.shape[1],n_feature), activation='tanh'))
GRU_GRU_reg.add(GRU(units=50, activation='tanh'))
GRU_GRU_reg.add(Dense(units=n_steps_out))


LSTM_LSTM_reg.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1],n_feature), activation='tanh'))
LSTM_LSTM_reg.add(LSTM(units=50, activation='tanh'))
LSTM_LSTM_reg.add(Dense(units=n_steps_out))


LSTM_GRU_reg.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1],n_feature), activation='tanh'))
LSTM_GRU_reg.add(GRU(units=50, activation='tanh'))
LSTM_GRU_reg.add(Dense(units=n_steps_out))


GRU_LSTM_reg.add(GRU(units=50, return_sequences=True, input_shape=(X_train.shape[1],n_feature), activation='tanh'))
GRU_LSTM_reg.add(LSTM(units=50, activation='tanh'))
GRU_LSTM_reg.add(Dense(units=n_steps_out))


# Compiling the RNNs
adam = optimizers.Adam(lr=0.01)

GRU_reg.compile(optimizer=adam,loss='mean_squared_error')
LSTM_reg.compile(optimizer=adam,loss='mean_squared_error')
GRU_GRU_reg.compile(optimizer=adam,loss='mean_squared_error')
LSTM_LSTM_reg.compile(optimizer=adam,loss='mean_squared_error')
LSTM_GRU_reg.compile(optimizer=adam,loss='mean_squared_error')
GRU_LSTM_reg.compile(optimizer=adam,loss='mean_squared_error')

INFO:plaidml:Opening device "llvm_cpu.0"


In [15]:
RnnModelDict = {'LSTM': LSTM_reg, 'GRU': GRU_reg, 'LSTM_LSTM': LSTM_LSTM_reg, 'GRU_GRU': GRU_GRU_reg, 
                'LSTM_GRU': LSTM_GRU_reg, 'GRU_LSTM': GRU_LSTM_reg}

X_test_24 = X_test[:24]
y_test_24 = y_test[:24]
rmse_df = pd.DataFrame(columns=['Model', 'train_rmse', 'valid_rmse', 'train_time'])

# RnnModelDict = {'LSTM_GRU': LSTM_GRU_reg}

In [16]:
for model in RnnModelDict:
    regressor = RnnModelDict[model]
    
    print('training start for', model)    
    start = time.process_time()
    regressor.fit(X_train,y_train,epochs=50,batch_size=32)
    train_time = round(time.process_time() - start, 2)
    
    print('results for training set')
    y_train_pred = regressor.predict(X_train)
    train_rmse = return_rmse(y_train,y_train_pred)
    
    print('results for valid set')
    y_valid_pred = regressor.predict(X_valid)
    valid_rmse = return_rmse(y_valid,y_valid_pred)
    
    
#     print('results for test set - 24 hours')
#     y_test_pred24 = regressor.predict(X_test_24)
#     plot_predictions(y_test_24,y_test_pred24)
#     test24_rmse = return_rmse(y_test_24,y_test_pred24)
    
    
    one_df = pd.DataFrame([[model, train_rmse, valid_rmse, train_time]], 
                          columns=['Model', 'train_rmse', 'valid_rmse', 'train_time'])
    rmse_df = pd.concat([rmse_df, one_df])

# save the rmse results 
rmse_df.to_csv('../rmse_24h_plus_time.csv')


training start for LSTM
Epoch 1/50
252414/252414 [==============================] - 151s 597us/step - loss: 7.2751e-04
Epoch 2/50
252414/252414 [==============================] - 119s 470us/step - loss: 4.5411e-04
Epoch 3/50
252414/252414 [==============================] - 128s 506us/step - loss: 3.8973e-04
Epoch 4/50
252414/252414 [==============================] - 131s 521us/step - loss: 3.6375e-04
Epoch 5/50
252414/252414 [==============================] - 133s 528us/step - loss: 3.2286e-04
Epoch 6/50
252414/252414 [==============================] - 136s 537us/step - loss: 3.2566e-04
Epoch 7/50
252414/252414 [==============================] - 136s 537us/step - loss: 3.2650e-04
Epoch 8/50
252414/252414 [==============================] - 138s 545us/step - loss: 3.1364e-04
Epoch 9/50
252414/252414 [==============================] - 149s 592us/step - loss: 2.3444e-04
Epoch 10/50
252414/252414 [==============================] - 146s 580us/step - loss: 2.2284e-04
Epoch 11/50
252414/252414

252414/252414 [==============================] - 300s 1ms/step - loss: 2.9031e-04
Epoch 21/50
252414/252414 [==============================] - 299s 1ms/step - loss: 2.5181e-04
Epoch 22/50
252414/252414 [==============================] - 299s 1ms/step - loss: 2.7564e-04
Epoch 23/50
252414/252414 [==============================] - 299s 1ms/step - loss: 2.9658e-04
Epoch 24/50
252414/252414 [==============================] - 301s 1ms/step - loss: 2.2751e-04
Epoch 25/50
252414/252414 [==============================] - 299s 1ms/step - loss: 2.9583e-04
Epoch 26/50
252414/252414 [==============================] - 299s 1ms/step - loss: 2.3922e-04
Epoch 27/50
252414/252414 [==============================] - 300s 1ms/step - loss: 2.2451e-04
Epoch 28/50
252414/252414 [==============================] - 300s 1ms/step - loss: 4.0813e-04
Epoch 29/50
252414/252414 [==============================] - 301s 1ms/step - loss: 3.9923e-04
Epoch 30/50
252414/252414 [==============================] - 301s 1ms/st